# CSV

In [1]:
import os
import csv

def check_and_fix_csv(file_path):
    header_needed = ["", "", "", "Tiempo (s)", "Voltaje (V)"]
    
    # Leer el archivo
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        lines = list(csv.reader(file))
    
    # Revisar si la fila 8 contiene el encabezado correcto
    if len(lines) >= 8 and lines[7] == header_needed:
        print(f"{file_path}: La fila 8 ya contiene el encabezado correcto.")
        return
    
    # Insertar el encabezado si no está
    lines.insert(7, header_needed)
    
    # Guardar el archivo modificado
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(lines)
    
    print(f"{file_path}: Encabezado agregado en la fila 8.")

# Procesar todos los archivos CSV en una carpeta
def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            check_and_fix_csv(file_path)

# Ruta de la carpeta con archivos CSV
folder_path = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k"  # Cambia esto según tu estructura de carpetas
process_folder(folder_path)


C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_165637_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_170004_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_170121_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_170640_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_170748_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k\250219_170854_Ch1.csv: La fila 8 ya contiene el encabezado correcto.
C:\Users\Nicolás Molina\OneDrive\Escrito

KeyboardInterrupt: 

# ANALISIS PICOS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.integrate import simpson

plt.close("all")

resistencia = 50  # 50 ohms o 1Mohm
N_adquisiciones = 5000  # Ajustar según corresponda
umbral_deteccion = -0.007  # Umbral para detectar un pico
umbral_integracion = -0.003  # Umbral para expandir la integración

def procesar_adquisicion(tiempo, voltaje, threshold_factor=-0.5):
    # Estimación de línea de base
    percentil_25 = np.percentile(voltaje, 25)
    percentil_75 = np.percentile(voltaje, 75)
    iqr = percentil_75 - percentil_25
    base_mask = (voltaje > percentil_25 - 1.5 * iqr) & (voltaje < percentil_75 + 1.5 * iqr)
    voltaje_base = voltaje[base_mask]
    media_base = np.mean(voltaje_base)
    voltaje_sin_base = voltaje - media_base

    # Detección del pico con umbral -0.007
    std_base = np.std(voltaje_base)
    threshold = media_base + threshold_factor * std_base
    indices_pico = np.where((voltaje_sin_base < threshold) & (voltaje_sin_base < umbral_deteccion))[0]
    if len(indices_pico) == 0:
        return None, None, None, None, None

    # Expandir la región de integración hasta -0.003
    inicio_pico, fin_pico = indices_pico[0], indices_pico[-1]
    
    # Expandir hacia la izquierda
    while inicio_pico > 0 and voltaje_sin_base[inicio_pico] < umbral_integracion:
        inicio_pico -= 1

    # Expandir hacia la derecha
    while fin_pico < len(voltaje_sin_base) - 1 and voltaje_sin_base[fin_pico] < umbral_integracion:
        fin_pico += 1

    # Datos para integración
    tiempo_pico = tiempo[inicio_pico:fin_pico+1]
    voltaje_pico = voltaje_sin_base[inicio_pico:fin_pico+1]
    carga = -simpson(voltaje_pico, tiempo_pico) / resistencia
    
    return tiempo, voltaje_sin_base, tiempo_pico, voltaje_pico, carga

def procesar_csv(archivo, tiempo_col='Tiempo (s)', voltaje_col='Voltaje (V)'):
    datos = pd.read_csv(archivo, skiprows=7)
    tiempo = datos[tiempo_col].values
    voltaje = datos[voltaje_col].values
    
    # Dividir en adquisiciones
    tamaño_adquisicion = len(tiempo) // N_adquisiciones
    cargas = []
    adquisiciones_con_pico = 0
    adquisiciones_sin_pico = 0
    
    plt.figure(figsize=(10, 5))
    for i in range(N_adquisiciones):
        inicio = i * tamaño_adquisicion
        fin = (i + 1) * tamaño_adquisicion
        tiempo_adq = tiempo[inicio:fin]
        voltaje_adq = voltaje[inicio:fin]
        
        tiempo_completo, voltaje_completo, tiempo_pico, voltaje_pico, carga = procesar_adquisicion(tiempo_adq, voltaje_adq)
        if carga is not None:
            cargas.append(carga)
            adquisiciones_con_pico += 1
            plt.plot(tiempo_pico, voltaje_pico, alpha=0.7)

    plt.xlabel("Tiempo (s)")
    plt.ylabel("Voltaje (V)")
    plt.title(f"Picos detectados - {os.path.basename(archivo)}")
    plt.legend([f"Picos detectados: {adquisiciones_con_pico}/{N_adquisiciones}"])
    plt.show()
    """
    # Gráfico sin zoom con todas las señales que contienen pico
    plt.figure(figsize=(10, 5))
    for i in range(N_adquisiciones):
        inicio = i * tamaño_adquisicion
        fin = (i + 1) * tamaño_adquisicion
        tiempo_adq = tiempo[inicio:fin]
        voltaje_adq = voltaje[inicio:fin]

        tiempo_completo, voltaje_completo, tiempo_pico, voltaje_pico, carga = procesar_adquisicion(tiempo_adq, voltaje_adq)
        if carga is not None:
            plt.plot(tiempo_completo, voltaje_completo, alpha=0.7)

    plt.xlabel("Tiempo (s)")
    plt.ylabel("Voltaje (V)")
    plt.title(f"Señales completas con pico - {os.path.basename(archivo)}")
    plt.legend([f"Picos detectados: {adquisiciones_con_pico}/{N_adquisiciones}"])
    plt.show()
    """
    print(f"En {adquisiciones_con_pico} adquisiciones se detectaron picos y en {adquisiciones_sin_pico} no.")
    
    return cargas

def procesar_carpeta(carpeta):
    cargas_totales = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.csv')]
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        cargas = procesar_csv(ruta)
        cargas_totales.extend(cargas)
    
    # Histograma de cargas con filtrado de valores atípicos
    cargas_arr = np.array(cargas_totales)
    Q1, Q3 = np.percentile(cargas_arr, [25, 75])
    IQR = Q3 - Q1
    lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    # Filtrar cargas dentro de este rango
    cargas_filtradas = cargas_arr[(cargas_arr >= lim_inf) & (cargas_arr <= lim_sup)]

    # Histograma con bins centrados en la región con más datos
    plt.figure()
    plt.hist(cargas_filtradas, bins=200, edgecolor='black', alpha=0.7)
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Cargas (sin valores atípicos)')
    plt.show()

    
    return cargas_totales

# Ruta de la carpeta con los CSVs
carpeta_csv = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k"
cargas = procesar_carpeta(carpeta_csv)




C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10700\2912737369.py:45: DeprecationWarning: You are passing x=[3.16e-08 3.17e-08 3.18e-08 3.19e-08 3.20e-08 3.21e-08 3.22e-08 3.23e-08
 3.24e-08 3.25e-08 3.26e-08 3.27e-08 3.28e-08 3.29e-08 3.30e-08 3.31e-08
 3.32e-08 3.33e-08 3.34e-08 3.35e-08 3.36e-08 3.37e-08 3.38e-08 3.39e-08
 3.40e-08 3.41e-08 3.42e-08 3.43e-08 3.44e-08 3.45e-08 3.46e-08 3.47e-08
 3.48e-08 3.49e-08 3.50e-08 3.51e-08 3.52e-08 3.53e-08 3.54e-08 3.55e-08
 3.56e-08 3.57e-08 3.58e-08 3.59e-08 3.60e-08 3.61e-08 3.62e-08 3.63e-08
 3.64e-08 3.65e-08 3.66e-08 3.67e-08 3.68e-08 3.69e-08 3.70e-08 3.71e-08
 3.72e-08] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  carga = -simpson(voltaje_pico, tiempo_pico) / resistencia
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10700\2912737369.py:45: DeprecationWarning: You are passing x=[2.91e-08 2.

En 181 adquisiciones se detectaron picos y en 0 no.


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10700\2912737369.py:45: DeprecationWarning: You are passing x=[2.900e-08 2.910e-08 2.920e-08 ... 1.642e-07 1.643e-07 1.644e-07] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  carga = -simpson(voltaje_pico, tiempo_pico) / resistencia
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10700\2912737369.py:45: DeprecationWarning: You are passing x=[3.24e-08 3.25e-08 3.26e-08 3.27e-08 3.28e-08 3.29e-08 3.30e-08 3.31e-08
 3.32e-08 3.33e-08 3.34e-08 3.35e-08 3.36e-08 3.37e-08 3.38e-08 3.39e-08
 3.40e-08 3.41e-08 3.42e-08 3.43e-08 3.44e-08 3.45e-08 3.46e-08 3.47e-08
 3.48e-08 3.49e-08 3.50e-08 3.51e-08 3.52e-08 3.53e-08 3.54e-08 3.55e-08
 3.56e-08 3.57e-08 3.58e-08 3.59e-08 3.60e-08 3.61e-08] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as posi

En 185 adquisiciones se detectaron picos y en 0 no.


# CODIGO BIEN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.integrate import simpson


#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')


plt.close("all")

resistencia = 50  # 50 ohms o 1Mohm
umbral_deteccion = -0.005  # Umbral para detectar un pico
umbral_integracion = -0.003  # Umbral para expandir la integración

def leer_numero_frames(archivo):
    with open(archivo, 'r') as f:
        for _ in range(6):  # Saltar las primeras 6 líneas
            next(f)
        linea_fastframe = next(f)  # Leer la línea que contiene "FastFrame Count"
    return int(linea_fastframe.split(',')[1])  # Extraer el número de frames

def procesar_adquisicion(tiempo, voltaje, threshold_factor=-0.5):
    percentil_25 = np.percentile(voltaje, 25)
    percentil_75 = np.percentile(voltaje, 75)
    iqr = percentil_75 - percentil_25
    base_mask = (voltaje > percentil_25 - 1.5 * iqr) & (voltaje < percentil_75 + 1.5 * iqr)
    voltaje_base = voltaje[base_mask]
    media_base = np.mean(voltaje_base)
    voltaje_sin_base = voltaje - media_base

    std_base = np.std(voltaje_base)
    threshold = media_base + threshold_factor * std_base
    indices_pico = np.where((voltaje_sin_base < threshold) & (voltaje_sin_base < umbral_deteccion))[0]
    if len(indices_pico) == 0:
        return None, None, None, None, None

    inicio_pico, fin_pico = indices_pico[0], indices_pico[-1]
    while inicio_pico > 0 and voltaje_sin_base[inicio_pico] < umbral_integracion:
        inicio_pico -= 1
    while fin_pico < len(voltaje_sin_base) - 1 and voltaje_sin_base[fin_pico] < umbral_integracion:
        fin_pico += 1

    tiempo_pico = tiempo[inicio_pico:fin_pico+1]
    voltaje_pico = voltaje_sin_base[inicio_pico:fin_pico+1]
    carga = -simpson(voltaje_pico, tiempo_pico) / resistencia
    
    return tiempo, voltaje_sin_base, tiempo_pico, voltaje_pico, carga

def procesar_csv(archivo, tiempo_col='Tiempo (s)', voltaje_col='Voltaje (V)'):
    N_adquisiciones = leer_numero_frames(archivo)  # Obtener el número de frames desde el archivo
    datos = pd.read_csv(archivo, skiprows=7)
    tiempo = datos[tiempo_col].values
    voltaje = datos[voltaje_col].values
    
    tamaño_adquisicion = len(tiempo) // N_adquisiciones  # Calcular el tamaño de cada adquisición
    cargas = []
    adquisiciones_con_pico = 0
    
    plt.figure(figsize=(10, 5))
    for i in range(N_adquisiciones):
        inicio = i * tamaño_adquisicion
        fin = (i + 1) * tamaño_adquisicion
        tiempo_adq = tiempo[inicio:fin]
        voltaje_adq = voltaje[inicio:fin]
        
        tiempo_completo, voltaje_completo, tiempo_pico, voltaje_pico, carga = procesar_adquisicion(tiempo_adq, voltaje_adq)
        if carga is not None:
            cargas.append(carga)
            adquisiciones_con_pico += 1
            plt.plot(tiempo_pico, voltaje_pico, alpha=0.7)

    plt.xlabel("Tiempo (s)")
    plt.ylabel("Voltaje (V)")
    plt.title(f"Picos detectados - {os.path.basename(archivo)}")
    plt.legend([f"Picos detectados: {adquisiciones_con_pico}/{N_adquisiciones}"])
    plt.show()
    
    print(f"En {adquisiciones_con_pico} adquisiciones se detectaron picos.")
    
    return cargas, N_adquisiciones

def procesar_carpeta(carpeta):
    cargas_totales = []
    N_adquisiciones_totales = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.csv')]
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        cargas, N_adquisiciones = procesar_csv(ruta)
        cargas_totales.extend(cargas)
        N_adquisiciones_totales.append(N_adquisiciones)
    
    cargas_arr = np.array(cargas_totales)
    Q1, Q3 = np.percentile(cargas_arr, [25, 75])
    IQR = Q3 - Q1
    lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    cargas_filtradas = cargas_arr[(cargas_arr >= lim_inf) & (cargas_arr <= lim_sup)]

    # Suma total de adquisiciones (disparos)
    total_disparos = sum(N_adquisiciones_totales)

    # Histograma de barras (número de cuentas)
    plt.figure(figsize=(12, 6))
    plt.hist(cargas_filtradas, bins=200, edgecolor='black', alpha=0.7, label='Histograma de barras')
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Número de cuentas')
    plt.title(f'Histograma de Cargas\nTotal de disparos: {total_disparos}')
    plt.legend()
    plt.show()

    # Histograma continuo (KDE)
    plt.figure(figsize=(12, 6))
    sns.kdeplot(cargas_filtradas, color='red', label='KDE (Histograma continuo)')
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Densidad')
    plt.title(f'Histograma Continuo (KDE)\nTotal de disparos: {total_disparos}')
    plt.legend()
    plt.show()
    
    return cargas_totales, N_adquisiciones_totales

carpeta_csv = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k"
cargas, N_adquisiciones_totales = procesar_carpeta(carpeta_csv)

En 455 adquisiciones se detectaron picos.
En 454 adquisiciones se detectaron picos.
En 420 adquisiciones se detectaron picos.
En 458 adquisiciones se detectaron picos.


KeyboardInterrupt: 

# analisis filtrando

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.integrate import simpson
from scipy.optimize import curve_fit
from scipy.stats import norm, cauchy

plt.close("all")

# Cambiar a 'inline' si quieres que los gráficos aparezcan en Jupyter
try:
    get_ipython().run_line_magic('matplotlib', 'qt5')
except:
    pass

# Variables globales para almacenar los resultados
resultados_histograma = {
    'cargas_totales': [],
    'cargas_filtradas': [],
    'bin_centers': None,
    'counts': None,
    'bin_edges': None,
    'total_disparos': 0
}

resistencia = 50  # 50 ohms o 1Mohm
umbral_deteccion = -0.002  # Umbral para detectar un pico
umbral_integracion = -0.003  # Umbral para expandir la integración

def leer_numero_frames(archivo):
    with open(archivo, 'r') as f:
        for _ in range(6):  # Saltar las primeras 6 líneas
            next(f)
        linea_fastframe = next(f)  # Leer la línea que contiene "FastFrame Count"
    return int(linea_fastframe.split(',')[1])  # Extraer el número de frames

def procesar_adquisicion(tiempo, voltaje, threshold_factor=-0.1):
    percentil_25 = np.percentile(voltaje, 25)
    percentil_75 = np.percentile(voltaje, 75)
    iqr = percentil_75 - percentil_25
    base_mask = (voltaje > percentil_25 - 1.5 * iqr) & (voltaje < percentil_75 + 1.5 * iqr)
    voltaje_base = voltaje[base_mask]
    media_base = np.mean(voltaje_base)
    voltaje_sin_base = voltaje - media_base

    std_base = np.std(voltaje_base)
    threshold = media_base + threshold_factor * std_base
    indices_pico = np.where((voltaje_sin_base < threshold) & (voltaje_sin_base < umbral_deteccion))[0]
    if len(indices_pico) == 0:
        return None, None, None, None, None

    inicio_pico, fin_pico = indices_pico[0], indices_pico[-1]
    while inicio_pico > 0 and voltaje_sin_base[inicio_pico] < umbral_integracion:
        inicio_pico -= 1
    while fin_pico < len(voltaje_sin_base) - 1 and voltaje_sin_base[fin_pico] < umbral_integracion:
        fin_pico += 1

    tiempo_pico = tiempo[inicio_pico:fin_pico+1]
    voltaje_pico = voltaje_sin_base[inicio_pico:fin_pico+1]
    carga = -simpson(voltaje_pico, tiempo_pico) / resistencia
    
    return tiempo, voltaje_sin_base, tiempo_pico, voltaje_pico, carga

def procesar_csv(archivo, tiempo_col='Tiempo (s)', voltaje_col='Voltaje (V)', plot_picos=True):
    N_adquisiciones = leer_numero_frames(archivo)  # Obtener el número de frames desde el archivo
    datos = pd.read_csv(archivo, skiprows=7)
    tiempo = datos[tiempo_col].values
    voltaje = datos[voltaje_col].values
    
    tamaño_adquisicion = len(tiempo) // N_adquisiciones  # Calcular el tamaño de cada adquisición
    cargas = []
    adquisiciones_con_pico = 0
    
    if plot_picos:
        plt.figure(figsize=(10, 5))
        for i in range(N_adquisiciones):
            inicio = i * tamaño_adquisicion
            fin = (i + 1) * tamaño_adquisicion
            tiempo_adq = tiempo[inicio:fin]
            voltaje_adq = voltaje[inicio:fin]
            
            tiempo_completo, voltaje_completo, tiempo_pico, voltaje_pico, carga = procesar_adquisicion(tiempo_adq, voltaje_adq)
            if carga is not None:
                cargas.append(carga)
                adquisiciones_con_pico += 1
                plt.plot(tiempo_pico, voltaje_pico, alpha=0.7)

        plt.xlabel("Tiempo (s)")
        plt.ylabel("Voltaje (V)")
        plt.title(f"Picos detectados - {os.path.basename(archivo)}")
        plt.legend([f"Picos detectados: {adquisiciones_con_pico}/{N_adquisiciones}"])
        plt.show()
    else:
        for i in range(N_adquisiciones):
            inicio = i * tamaño_adquisicion
            fin = (i + 1) * tamaño_adquisicion
            tiempo_adq = tiempo[inicio:fin]
            voltaje_adq = voltaje[inicio:fin]
            
            _, _, _, _, carga = procesar_adquisicion(tiempo_adq, voltaje_adq)
            if carga is not None:
                cargas.append(carga)
                adquisiciones_con_pico += 1
    
    print(f"En {adquisiciones_con_pico} adquisiciones se detectaron picos.")
    
    return cargas, N_adquisiciones

def procesar_carpeta(carpeta, plot_picos=True, guardar_datos=True, n_bins=200):
    cargas_totales = []
    N_adquisiciones_totales = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.csv')]
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        cargas, N_adquisiciones = procesar_csv(ruta, plot_picos=plot_picos)
        cargas_totales.extend(cargas)
        N_adquisiciones_totales.append(N_adquisiciones)
    
    cargas_arr = np.array(cargas_totales)
    Q1, Q3 = np.percentile(cargas_arr, [25, 75])
    IQR = Q3 - Q1
    lim_inf, lim_sup = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    cargas_filtradas = cargas_arr[(cargas_arr >= lim_inf) & (cargas_arr <= lim_sup)]

    # Suma total de adquisiciones (disparos)
    total_disparos = sum(N_adquisiciones_totales)

    # Histograma de barras (número de cuentas)
    plt.figure(figsize=(12, 6))
    counts, bin_edges, _ = plt.hist(cargas_filtradas, bins=n_bins, edgecolor='black', alpha=0.7, label='Histograma de barras')
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Centros de los bins
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Número de cuentas (escala logarítmica)')
    plt.title(f'Histograma de Cargas\nTotal de disparos: {total_disparos}')
    plt.legend()
    plt.show()

    # Excluir valores menores a 0.2 Coulombs para el filtrado posterior
    mask = bin_centers >= 0.2 * 10**(-12)
    bin_centers_filtrados = bin_centers[mask]
    counts_filtrados = counts[mask]


    
    # Guardar los resultados para acceso posterior
    if guardar_datos:
        resultados_histograma['cargas_totales'] = cargas_totales
        resultados_histograma['cargas_filtradas'] = cargas_filtradas
        resultados_histograma['bin_centers'] = bin_centers
        resultados_histograma['counts'] = counts
        resultados_histograma['bin_edges'] = bin_edges
        resultados_histograma['total_disparos'] = total_disparos
        resultados_histograma['bin_centers_filtrados'] = bin_centers_filtrados
        resultados_histograma['counts_filtrados'] = counts_filtrados
    
    return cargas_totales, N_adquisiciones_totales, resultados_histograma

def guardar_resultados(archivo_salida='resultados_histograma.npz'):
    """
    Guarda los resultados del histograma en un archivo para acceso posterior.
    """
    np.savez(archivo_salida, 
             cargas_totales=resultados_histograma['cargas_totales'],
             cargas_filtradas=resultados_histograma['cargas_filtradas'],
             bin_centers=resultados_histograma['bin_centers'],
             counts=resultados_histograma['counts'],
             bin_edges=resultados_histograma['bin_edges'],
             total_disparos=resultados_histograma['total_disparos'])
    print(f"Resultados guardados en {archivo_salida}")

def cargar_resultados(archivo='resultados_histograma.npz'):
    """
    Carga los resultados del histograma desde un archivo.
    """
    data = np.load(archivo)
    resultados_histograma['cargas_totales'] = data['cargas_totales']
    resultados_histograma['cargas_filtradas'] = data['cargas_filtradas']
    resultados_histograma['bin_centers'] = data['bin_centers']
    resultados_histograma['counts'] = data['counts']
    resultados_histograma['bin_edges'] = data['bin_edges']
    resultados_histograma['total_disparos'] = data['total_disparos']
    print("Resultados cargados con éxito")
    return resultados_histograma

def mostrar_histograma(escala_logaritmica=False):
    """
    Muestra el histograma usando los datos almacenados en resultados_histograma.
    """
    if not resultados_histograma['counts'] is None:
        plt.figure(figsize=(12, 6))
        plt.bar(resultados_histograma['bin_centers'], 
                resultados_histograma['counts'], 
                width=resultados_histograma['bin_edges'][1] - resultados_histograma['bin_edges'][0],
                edgecolor='black', alpha=0.7, label='Histograma de barras')
        plt.xlabel('Carga (Coulombs)')
        
        if escala_logaritmica:
            plt.yscale('log')
            plt.ylabel('Número de cuentas (escala logarítmica)')
        else:
            plt.ylabel('Número de cuentas')
            
        plt.title(f'Histograma de Cargas\nTotal de disparos: {resultados_histograma["total_disparos"]}')
        plt.legend()
        plt.show()
    else:
        print("No hay datos de histograma disponibles. Ejecuta primero procesar_carpeta().")



# Ruta a la carpeta de archivos CSV (cámbiala según tu necesidad)
carpeta_csv = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\8,66v 10k"

# Para ejecutar el procesamiento:
cargas, N_adquisiciones_totales, histograma_data = procesar_carpeta(carpeta_csv, plot_picos=False)

# Para guardar los resultados después de procesar:
guardar_resultados("exppp.npz")

# Para cargar resultados previamente guardados:
# histograma_data = cargar_resultados("mi_experimento_resultados.npz")

# Para mostrar el histograma sin volver a procesar:
# mostrar_histograma(escala_logaritmica=True)


En 4986 adquisiciones se detectaron picos.
En 4998 adquisiciones se detectaron picos.
En 4723 adquisiciones se detectaron picos.
En 4997 adquisiciones se detectaron picos.
En 4991 adquisiciones se detectaron picos.
En 4994 adquisiciones se detectaron picos.
En 4994 adquisiciones se detectaron picos.
En 4979 adquisiciones se detectaron picos.
En 4996 adquisiciones se detectaron picos.
En 4992 adquisiciones se detectaron picos.
En 4989 adquisiciones se detectaron picos.
En 4981 adquisiciones se detectaron picos.
En 4998 adquisiciones se detectaron picos.
En 4987 adquisiciones se detectaron picos.
En 4995 adquisiciones se detectaron picos.
En 4993 adquisiciones se detectaron picos.
En 4998 adquisiciones se detectaron picos.
En 4995 adquisiciones se detectaron picos.
En 4988 adquisiciones se detectaron picos.
En 4991 adquisiciones se detectaron picos.
En 4990 adquisiciones se detectaron picos.
En 4991 adquisiciones se detectaron picos.
En 9997 adquisiciones se detectaron picos.
Resultados 

In [64]:
datos = cargar_resultados("mi_experimento_resultados.npz")
plot_histograma()

KeyError: 'cargas_picos is not a file in the archive'

# agarrar histogramas

In [65]:

# Para cargar resultados previamente guardados:
histograma_data = cargar_resultados("mi_experimento_resultados.npz")

# Para mostrar el histograma sin volver a procesar:
mostrar_histograma(escala_logaritmica=False)

print(type(histograma_data))
# Si es un diccionario:
print(histograma_data.keys())

# Si es un array:
#print(histograma_data.shape)



KeyError: 'cargas_picos is not a file in the archive'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'qt5')

# Cargar los datos
histograma_data = np.load("mi_experimento_resultados.npz", allow_pickle=True)

# Número de bins a eliminar
n = 0  

# Extraer los valores correctos
counts = histograma_data['counts']
bin_edges = histograma_data['bin_edges']

# Asegurar que las dimensiones se mantengan correctas al recortar
counts = counts[n:]
bin_edges = bin_edges[n:]  # Se corta desde n para mantener la relación correcta
plt.figure()
# Graficar el histograma
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.xlabel("Carga (Coulombs)")
plt.ylabel("Cuentas")
plt.title("Histograma ajustado")
plt.show()


TypeError: load() got an unexpected keyword argument 'escala_logaritmica'

# ajuste posta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erfc
plt.close("all")
# Definir la función S(x)
w=0.85

n=0
def S(x, alpha, Q, sigma):
      # Valor fijo
      # Valor fijo

    H = np.where(x >= 0, 1.0, 0.0)
    
    exp_part = w * alpha * np.exp(-alpha * x) * H
    g_N = 0.5 * erfc(-Q / (np.sqrt(2) * sigma))
    if g_N == 0:
      g_N = 1e-10  # Evitar divisiones por cero

    gaussian_part = (1 - w) * (1 / np.sqrt(2 * np.pi * sigma**2)) * np.exp(-(x - Q)**2 / (2 * sigma**2)) * H / g_N

    return exp_part + gaussian_part

# Cargar datos del histograma
bin_centers = histograma_data['bin_centers']
counts = histograma_data['counts']

counts= counts[n:]

bin_centers = bin_centers[n:]


 

# Ajuste de curva
try:
    popt, pcov = curve_fit(S, bin_centers,counts,maxfev=10000)

    # Extraer parámetros ajustados
    alpha, Q, sigma = popt
      # Fijo
      
    g_N = 0.5 * erfc(-Q / (np.sqrt(2) * sigma))

    # Cálculo de los parámetros adicionales
    kappa = (1 / (g_N * np.sqrt(2 * np.pi))) * sigma * np.exp(-Q**2 / (2 * sigma**2))
    Q_g = Q + kappa
    sigma_g2 = sigma**2 - (Q + kappa) * kappa
    Q_s = (w / alpha) + (1 - w) * Q_g
    sigma_s2 = (w / alpha**2) + (1 - w) * sigma_g2 + w * (1 - w) * (Q_g - 1 / alpha)**2

    # Graficar resultados
    plt.figure(figsize=(12, 6))
    plt.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], 
            edgecolor='black', alpha=0.7, label='Histograma')
    plt.plot(bin_centers, S(bin_centers, *popt), color='red', label='Ajuste S(x)')
    plt.xlabel('x')
    plt.ylabel('Cuentas')
    plt.title('Ajuste de S(x) con w=0.85 fijo')
    plt.legend()
    plt.show()

    # Mostrar resultados
    print("Parámetros ajustados:")
    print(f"Exponencial: alpha = {alpha}")
    print(f"Gaussiana: Q = {Q}, sigma = {sigma}")
    print(f"\nFactor g_N (normalización): {g_N:.4f}")
    print("\nParámetros adicionales calculados:")
    print(f"Kappa = {kappa}")
    print(f"Q_g = {Q_g}")
    print(f"sigma_g^2 = {sigma_g2}")
    print(f"Q_s = {Q_s}")
    print(f"sigma_s^2 = {sigma_s2}")

except Exception as e:
    print(f"Error en el ajuste: {e}")
    print("Revisa los datos o los parámetros iniciales.")


Parámetros ajustados:
Exponencial: alpha = 60.38196289945241
Gaussiana: Q = -1821.096086665179, sigma = -9707.05545930974

Factor g_N (normalización): 0.5744

Parámetros adicionales calculados:
Kappa = -6624.226593542066
Q_g = -8445.322680207246
sigma_g^2 = 38283194.60084224
Q_s = -1266.7843249796103
sigma_s^2 = 14836257.940501992


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf, erfc
import math

plt.close("all")
p_E = 0.85

import numpy as np
from scipy.special import erf, erfc
import math

def complex_function(x, λ, A, μ_ped, σ_ped, σ, μ, μ_1, σ_1):
    """
    Función ajustada exactamente a la ecuación mostrada en la imagen
    """
    p_E = 0.85  # Valor fijo según el código original
    g_N = 0.5 * (1 + erf(μ/(np.sqrt(2)*σ)))
    # Término n=0 (término gaussiano pedestal)
    term0 = (np.exp(-λ) / (np.sqrt(2*np.pi)*σ_ped)) * np.exp(-(x-μ_ped)**2/(2*σ_ped**2))
    
    # Término n=1 (componente principal)
    # Primera parte exponencial con erf
    exp_part = (σ_ped**2 - 2*A*(x - μ_ped))/(2*A**2)
    erf_part = (A*(x - μ_ped) - σ_ped**2)/(np.sqrt(2)*σ_ped*A)
    part1 = (p_E/(2*A)) * np.exp(exp_part) * (1 + erf(erf_part))
    
    # Segunda parte gaussiana
    part2 = ((1-p_E)/g_N) * (1/np.sqrt(2*np.pi*(σ**2 + σ_ped**2))) * np.exp(-(x - μ - μ_ped)**2/(2*(σ**2 + σ_ped**2)))
    
    term1 = np.exp(-λ) * λ * (part1 + part2)
    
    # Términos n=2 a 4 (fotones múltiples)
    terms_n = []
    for n in range(2, 5):
        variance = n*σ_1**2 + σ_ped**2
        mean = n*μ_1 + μ_ped
        term_n = (np.exp(-λ)*λ**n/math.factorial(n)) * (1/np.sqrt(2*np.pi*variance)) * np.exp(-(x - mean)**2/(2*variance))
        terms_n.append(term_n)
    
    return term0 + term1 + sum(terms_n)
# Cargar datos del histograma
bin_centers = histograma_data['bin_centers']
counts = histograma_data['counts']

# Eliminar posibles puntos iniciales si es necesario
n = 0
counts = counts[n:]
bin_centers = bin_centers[n:]

# Valores iniciales para los parámetros
#initial_guess = [1.0, 0.1, np.mean(bin_centers[:15]), np.std(bin_centers[:15]), 1.0, 2e-12, 2e-12]

# Límites para los parámetros (lower_bounds, upper_bounds)
lower_bounds = [1e-20, 1e-20, 1e-16, 1e-16, 1e-20, 1e-13, 1e-13,0]
upper_bounds = [np.inf, np.inf, 1e-12, 1e-12, np.inf, 1e-11, 1e-11,np.inf]

# Ajuste de curva con bounds específicos
try:
    popt, pcov = curve_fit(complex_function, bin_centers, counts,
                          
                          maxfev=10000,
                          bounds=(lower_bounds, upper_bounds))
    
    # Extraer parámetros ajustados
    λ_fit, A_fit, μ_ped_fit, σ_ped_fit, σ_fit, μ_1_fit, σ_1_fit ,μ_fit= popt
    
    # Graficar resultados
    plt.figure(figsize=(12, 6))
    plt.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], 
            edgecolor='black', alpha=0.7, label='Histograma')
    
    x_fine = np.linspace(min(bin_centers), max(bin_centers), 500)
    plt.plot(x_fine, complex_function(x_fine, *popt), color='red', label='Ajuste completo')
    
    plt.xlabel('x')
    plt.ylabel('Cuentas')
    plt.title('Ajuste de la función compleja con bounds específicos')
    plt.legend()
    plt.show()

    # Mostrar resultados en notación científica
    print("Parámetros ajustados:")
    print(f"λ (Poisson) = {λ_fit:.4e}")
    print(f"A = {A_fit:.4e}")
    print(f"μ_ped = {μ_ped_fit:.4e}")
    print(f"σ_ped = {σ_ped_fit:.4e}")
    print(f"σ = {σ_fit:.4e}")
    print(f"μ_1 = {μ_1_fit:.4e}")
    print(f"σ_1 = {σ_1_fit:.4e}")
    print(f"μ = {μ_fit:.4e}")
except Exception as e:
    print(f"Error en el ajuste: {e}")
    print("Revisa los datos o los parámetros iniciales.")

Parámetros ajustados:
λ (Poisson) = 2.2357e+01
A = 1.1582e+05
μ_ped = 8.7298e-13
σ_ped = 7.0838e-13
σ = 1.3985e+02
μ_1 = 5.1495e-12
σ_1 = 5.0500e-12
μ = 7.6694e+05


In [33]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf
import math

class RobustFitter:
    def __init__(self, p_E=0.85, max_iter=10000):
        self.p_E = p_E
        self.max_iter = max_iter
        self.parameters = None
        self.errors = None
        
    def model_function(self, x, λ, A, μ_ped, σ_ped, σ, μ, μ_1, σ_1):
        """Función física completa con todos los términos"""
        try:
            g_N = 0.5 * (1 + erf(μ/(np.sqrt(2)*σ)))
            
            # Término pedestal (n=0)
            term0 = (np.exp(-λ) / (np.sqrt(2*np.pi)*σ_ped)) * np.exp(-(x-μ_ped)**2/(2*σ_ped**2))
            
            # Término principal (n=1)
            exp_part = (σ_ped**2 - 2*A*(x - μ_ped))/(2*A**2)
            erf_part = (A*(x - μ_ped) - σ_ped**2)/(np.sqrt(2)*σ_ped*A)
            part1 = (self.p_E/(2*A)) * np.exp(exp_part) * (1 + erf(erf_part))
            
            part2 = ((1-self.p_E)/g_N) * (1/np.sqrt(2*np.pi*(σ**2 + σ_ped**2))) * \
                    np.exp(-(x - μ - μ_ped)**2/(2*(σ**2 + σ_ped**2)))
            
            term1 = np.exp(-λ) * λ * (part1 + part2)
            
            # Términos de fotones múltiples (n=2 a 4)
            terms_n = []
            for n in range(2, 5):
                variance = n*σ_1**2 + σ_ped**2
                mean = n*μ_1 + μ_ped
                term_n = (np.exp(-λ)*λ**n/math.factorial(n)) * \
                        (1/np.sqrt(2*np.pi*variance)) * np.exp(-(x - mean)**2/(2*variance))
                terms_n.append(term_n)
            
            return term0 + term1 + sum(terms_n)
        except:
            return np.full_like(x, np.nan)  # Devuelve array de NaN si hay error numérico

    def fit(self, x_data, y_data, initial_guess=None):
        """Método de ajuste robustecido"""
        # Preprocesamiento manual (sin sklearn)
        x_scale = np.std(x_data)
        y_scale = np.std(y_data)
        x_norm = x_data / x_scale
        y_norm = y_data / y_scale
        
        # Límites para parámetros normalizados
        bounds = (
            [1e-10, 1e-10, -10, 1e-10, 1e-10, -10, -10, 1e-10],  # Límites inferiores
            [10, 10, 10, 10, 10, 10, 10, 10]    # Límites superiores
        )
        
        # Estimación inicial inteligente si no se provee
        if initial_guess is None:
            initial_guess = [
                1.0,    # λ
                0.1,    # A
                np.mean(x_norm[y_norm > 0.5*np.max(y_norm)]),  # μ_ped
                np.std(x_norm)/2,   # σ_ped
                np.std(x_norm),     # σ
                0.5,    # μ
                1.0,    # μ_1
                0.5     # σ_1
            ]
        
        try:
            # Ajuste con parámetros normalizados
            popt_norm, pcov_norm = curve_fit(
                self.model_function,
                x_norm, y_norm,
                p0=initial_guess,
                maxfev=self.max_iter,
                bounds=bounds,
                method='trf'  # Método más robusto
            )
            
            # Re-escalar parámetros a unidades originales
            self.parameters = {
                'λ': popt_norm[0],
                'A': popt_norm[1] * y_scale,
                'μ_ped': popt_norm[2] * x_scale,
                'σ_ped': popt_norm[3] * x_scale,
                'σ': popt_norm[4] * x_scale,
                'μ': popt_norm[5] * x_scale,
                'μ_1': popt_norm[6] * x_scale,
                'σ_1': popt_norm[7] * x_scale
            }
            
            # Calcular errores (aproximados)
            with np.errstate(invalid='ignore'):
                perr = np.sqrt(np.diag(pcov_norm))
                self.errors = {
                    'λ_err': perr[0],
                    'A_err': perr[1] * y_scale,
                    'μ_ped_err': perr[2] * x_scale,
                    'σ_ped_err': perr[3] * x_scale,
                    'σ_err': perr[4] * x_scale,
                    'μ_err': perr[5] * x_scale,
                    'μ_1_err': perr[6] * x_scale,
                    'σ_1_err': perr[7] * x_scale
                }
            
            return True
        except Exception as e:
            print(f"Error en el ajuste: {str(e)}")
            self.parameters = None
            self.errors = None
            return False

    def plot_results(self, x_data, y_data):
        """Visualización de resultados"""
        if self.parameters is None:
            print("No hay parámetros ajustados para graficar")
            return
            
        plt.figure(figsize=(12, 6))
        
        # Datos originales
        plt.bar(x_data, y_data, width=x_data[1]-x_data[0], 
                alpha=0.5, label='Datos experimentales')
        
        # Ajuste completo
        x_fine = np.linspace(min(x_data), max(x_data), 500)
        y_fit = self.model_function(x_fine, **self.parameters)
        plt.plot(x_fine, y_fit, 'r-', lw=2, label='Ajuste completo')
        
        # Componentes individuales (opcional)
        params = self.parameters
        try:
            # Término pedestal
            term0 = (np.exp(-params['λ']) / (np.sqrt(2*np.pi)*params['σ_ped'])) * \
                   np.exp(-(x_fine-params['μ_ped'])**2/(2*params['σ_ped']**2))
            plt.plot(x_fine, term0, 'g--', label='Término pedestal')
            
            # Término principal
            g_N = 0.5 * (1 + erf(params['μ']/(np.sqrt(2)*params['σ'])))
            exp_part = (params['σ_ped']**2 - 2*params['A']*(x_fine - params['μ_ped']))/(2*params['A']**2)
            erf_part = (params['A']*(x_fine - params['μ_ped']) - params['σ_ped']**2)/(np.sqrt(2)*params['σ_ped']*params['A'])
            part1 = (self.p_E/(2*params['A'])) * np.exp(exp_part) * (1 + erf(erf_part))
            part2 = ((1-self.p_E)/g_N) * (1/np.sqrt(2*np.pi*(params['σ']**2 + params['σ_ped']**2))) * \
                   np.exp(-(x_fine - params['μ'] - params['μ_ped'])**2/(2*(params['σ']**2 + params['σ_ped']**2)))
            term1 = np.exp(-params['λ']) * params['λ'] * (part1 + part2)
            plt.plot(x_fine, term1, 'b--', label='Término principal')
        except:
            pass
        
        plt.xlabel('Carga (Coulombs)')
        plt.ylabel('Cuentas normalizadas')
        plt.title('Ajuste del modelo físico')
        plt.legend()
        plt.show()

# Ejemplo de uso
if __name__ == "__main__":
    # Datos de ejemplo (simulados)
    x_data = np.linspace(0, 10, 100)
    y_data = np.exp(-x_data/2) + 0.5*np.exp(-(x_data-2)**2/0.5) + 0.3*np.exp(-(x_data-4)**2/0.5) + np.random.normal(0, 0.05, 100)
    
    # Crear y ajustar el modelo
    fitter = RobustFitter(p_E=0.85)
    if fitter.fit(x_data, y_data):
        print("Ajuste exitoso!")
        print("Parámetros ajustados:")
        for name, value in fitter.parameters.items():
            print(f"{name}: {value:.4e} ± {fitter.errors.get(name+'_err', 'N/A'):.4e}")
        
        # Graficar resultados
        fitter.plot_results(x_data, y_data)
    else:
        print("El ajuste falló. Intenta con diferentes valores iniciales o revisa los datos.")

Ajuste exitoso!
Parámetros ajustados:
λ: 1.2777e+00 ± 1.2985e-01
A: 1.0433e-01 ± 3.8589e-02
μ_ped: 3.8934e+00 ± 4.3933e-02
σ_ped: 3.6290e-01 ± 4.1961e-02
σ: 1.5870e+00 ± 4.1734e-02
μ: -3.4377e+00 ± 1.1689e-01
μ_1: -7.7320e-01 ± 4.5253e-02
σ_1: 2.9158e-10 ± 7.7885e+05


# ajustes chotos

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Definir la función compuesta (exponencial + 3 gaussianas)
def funcion_compuesta(x, A_exp, lambd, A1, mu1, sigma1, A2, mu2, sigma2, A3, mu3, sigma3):
    # Parte exponencial
    exp_part = A_exp * np.exp(-lambd * x)
    
    # Parte gaussiana (3 gaussianas)
    gauss1 = A1 * np.exp(-(x - mu1)**2 / (2 * sigma1**2))
    gauss2 = A2 * np.exp(-(x - mu2)**2 / (2 * sigma2**2))
    gauss3 = A3 * np.exp(-(x - mu3)**2 / (2 * sigma3**2))
    
    return exp_part + gauss1 + gauss2 + gauss3

# Cargar los datos del histograma
bin_centers = histograma_data['bin_centers']
counts = histograma_data['counts']

# Parámetros iniciales (estimaciones razonables)
# Exponencial
A_exp_initial = max(counts)  # Amplitud inicial de la exponencial
lambd_initial = 1.0  # Tasa de decaimiento inicial

# Gaussianas (estimaciones basadas en el histograma)
A1_initial = max(counts)  # Amplitud de la primera gaussiana
mu1_initial = bin_centers[np.argmax(counts)]  # Media de la primera gaussiana
sigma1_initial = (bin_centers[-1] - bin_centers[0]) / 10  # Desviación estándar de la primera gaussiana

A2_initial = max(counts) / 2  # Amplitud de la segunda gaussiana
mu2_initial = mu1_initial + (bin_centers[-1] - bin_centers[0]) / 3  # Media de la segunda gaussiana
sigma2_initial = sigma1_initial  # Desviación estándar de la segunda gaussiana

A3_initial = max(counts) / 3  # Amplitud de la tercera gaussiana
mu3_initial = mu2_initial + (bin_centers[-1] - bin_centers[0]) / 3  # Media de la tercera gaussiana
sigma3_initial = sigma1_initial  # Desviación estándar de la tercera gaussiana

# Parámetros iniciales combinados
p0 = [A_exp_initial, lambd_initial, 
      A1_initial, mu1_initial, sigma1_initial, 
      A2_initial, mu2_initial, sigma2_initial, 
      A3_initial, mu3_initial, sigma3_initial]

# Ajustar la función compuesta al histograma
try:
    popt, pcov = curve_fit(funcion_compuesta, bin_centers, counts, p0=p0, maxfev=10000)
    
    # Graficar el histograma y el ajuste
    plt.figure(figsize=(12, 6))
    plt.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], edgecolor='black', alpha=0.7, label='Histograma')
    plt.plot(bin_centers, funcion_compuesta(bin_centers, *popt), color='red', label='Ajuste Compuesto')
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Número de cuentas')
    plt.title('Ajuste de Exponencial + 3 Gaussianas al Histograma')
    plt.legend()
    plt.show()

    # Mostrar los parámetros ajustados
    print("Parámetros ajustados:")
    print(f"Exponencial: A_exp = {popt[0]}, lambda = {popt[1]}")
    print(f"Gaussiana 1: A1 = {popt[2]}, mu1 = {popt[3]}, sigma1 = {popt[4]}")
    print(f"Gaussiana 2: A2 = {popt[5]}, mu2 = {popt[6]}, sigma2 = {popt[7]}")
    print(f"Gaussiana 3: A3 = {popt[8]}, mu3 = {popt[9]}, sigma3 = {popt[10]}")
except Exception as e:
    print(f"Error en el ajuste: {e}")
    print("Revisa los datos o los parámetros iniciales.")

Parámetros ajustados:
Exponencial: A_exp = 2.6051304477056068, lambda = 1.0
Gaussiana 1: A1 = 83.45555223709412, mu1 = 2.4378436194370255e-13, sigma1 = 1.1731894166988193e-13
Gaussiana 2: A2 = 41.70871148908418, mu2 = 8.562660484712883e-13, sigma2 = 5.803995710950007e-13
Gaussiana 3: A3 = 68.02120759244819, mu3 = 2.096222544025596e-12, sigma3 = 1.1459362177330534e-12


C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_10024\2812245497.py:47: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(funcion_compuesta, bin_centers, counts, p0=p0, maxfev=10000)


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Definir la función exponencial decreciente
def exponencial_decreciente(x, A, lambd):
    return A * np.exp(-lambd * x)

# Definir una función compuesta (exponencial + 3 gaussianas)
def funcion_compuesta(x, A_exp, lambd, A1, mu1, sigma1, A2, mu2, sigma2, A3, mu3, sigma3):
    exp_part = A_exp * np.exp(-lambd * x)
    gauss1 = A1 * np.exp(-(x - mu1)**2 / (2 * sigma1**2))
    gauss2 = A2 * np.exp(-(x - mu2)**2 / (2 * sigma2**2))
    gauss3 = A3 * np.exp(-(x - mu3)**2 / (2 * sigma3**2))
    return exp_part + gauss1 + gauss2 + gauss3

# Cargar los datos del histograma
bin_centers = histograma_data['bin_centers']
counts = histograma_data['counts']

# Ignorar los primeros datos de carga (por ejemplo, los primeros 10 bins)
umbral = 2  # Número de bins a ignorar
bin_centers_filtrados = bin_centers[umbral:]
counts_filtrados = counts[umbral:]

# Paso 1: Ajustar la exponencial decreciente a los datos filtrados
try:
    # Parámetros iniciales para la exponencial
    A_initial = max(counts_filtrados)  # Amplitud inicial
    lambd_initial = 1.0  # Tasa de decaimiento inicial

    # Ajustar la exponencial
    popt_exp, _ = curve_fit(exponencial_decreciente, bin_centers_filtrados, counts_filtrados, 
                            p0=[A_initial, lambd_initial])
    
    # Graficar el ajuste de la exponencial
    plt.figure(figsize=(12, 6))
    plt.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], edgecolor='black', alpha=0.7, label='Histograma')
    plt.plot(bin_centers_filtrados, exponencial_decreciente(bin_centers_filtrados, *popt_exp), '--', color='red', label='Ajuste Exponencial (datos filtrados)')
    plt.xlabel('Carga (Coulombs)')
    plt.ylabel('Número de cuentas')
    plt.title('Ajuste de la Exponencial Decreciente (Ignorando Primeros Datos)')
    plt.legend()
    plt.show()

    print("Parámetros ajustados de la exponencial decreciente:")
    print(f"A = {popt_exp[0]}, lambda = {popt_exp[1]}")
except Exception as e:
    print(f"Error en el ajuste de la exponencial: {e}")
    popt_exp = None  # Si falla, definir popt_exp como None

# Paso 2: Ajustar la función compuesta (exponencial + 3 gaussianas)
if popt_exp is not None:  # Solo continuar si el ajuste de la exponencial fue exitoso
    try:
        # Parámetros iniciales para la función compuesta
        A_exp_initial = popt_exp[0]  # Amplitud de la exponencial
        lambd_initial = popt_exp[1]  # Tasa de decaimiento de la exponencial

        # Parámetros iniciales para las gaussianas
        A1_initial = max(counts_filtrados)  # Amplitud de la primera gaussiana
        mu1_initial = bin_centers_filtrados[np.argmax(counts_filtrados)]  # Media de la primera gaussiana
        sigma1_initial = (bin_centers_filtrados[-1] - bin_centers_filtrados[0]) / 10  # Desviación estándar de la primera gaussiana

        A2_initial = max(counts_filtrados) / 2  # Amplitud de la segunda gaussiana
        mu2_initial = mu1_initial + (bin_centers_filtrados[-1] - bin_centers_filtrados[0]) / 3  # Media de la segunda gaussiana
        sigma2_initial = (bin_centers_filtrados[-1] - bin_centers_filtrados[0]) / 10  # Desviación estándar de la segunda gaussiana

        A3_initial = max(counts_filtrados) / 3  # Amplitud de la tercera gaussiana
        mu3_initial = mu2_initial + (bin_centers_filtrados[-1] - bin_centers_filtrados[0]) / 3  # Media de la tercera gaussiana
        sigma3_initial = (bin_centers_filtrados[-1] - bin_centers_filtrados[0]) / 10  # Desviación estándar de la tercera gaussiana

        # Parámetros iniciales combinados
        p0 = [A_exp_initial, lambd_initial, 
              A1_initial, mu1_initial, sigma1_initial, 
              A2_initial, mu2_initial, sigma2_initial, 
              A3_initial, mu3_initial, sigma3_initial]

        # Ajustar la función compuesta
        popt_comp, _ = curve_fit(funcion_compuesta, bin_centers_filtrados, counts_filtrados, p0=p0, maxfev=10000)
        
        # Graficar el histograma completo y el ajuste compuesto
        plt.figure(figsize=(12, 6))
        plt.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], edgecolor='black', alpha=0.7, label='Histograma')
        plt.plot(bin_centers_filtrados, funcion_compuesta(bin_centers_filtrados, *popt_comp), '--', color='red', label='Ajuste Compuesto')
        plt.xlabel('Carga (Coulombs)')
        plt.ylabel('Número de cuentas')
        plt.title('Ajuste de Exponencial + 3 Gaussianas (Ignorando Primeros Datos)')
        plt.legend()
        plt.show()

        print("Parámetros ajustados de la función compuesta:")
        print(f"Exponencial: A_exp = {popt_comp[0]}, lambda = {popt_comp[1]}")
        print(f"Gaussiana 1: A1 = {popt_comp[2]}, mu1 = {popt_comp[3]}, sigma1 = {popt_comp[4]}")
        print(f"Gaussiana 2: A2 = {popt_comp[5]}, mu2 = {popt_comp[6]}, sigma2 = {popt_comp[7]}")
        print(f"Gaussiana 3: A3 = {popt_comp[8]}, mu3 = {popt_comp[9]}, sigma3 = {popt_comp[10]}")
    except Exception as e:
        print(f"Error en el ajuste de la función compuesta: {e}")
else:
    print("No se pudo ajustar la exponencial, por lo que no se puede continuar con el ajuste compuesto.")

C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_10024\3346937144.py:33: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_exp, _ = curve_fit(exponencial_decreciente, bin_centers_filtrados, counts_filtrados,
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_10024\3346937144.py:79: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_comp, _ = curve_fit(funcion_compuesta, bin_centers_filtrados, counts_filtrados, p0=p0, maxfev=10000)


Parámetros ajustados de la exponencial decreciente:
A = 53.41919192910766, lambda = 1.0
Parámetros ajustados de la función compuesta:
Exponencial: A_exp = 1.5748765903618895, lambda = 1.0
Gaussiana 1: A1 = 73.07326732507427, mu1 = 2.2589375272575394e-13, sigma1 = 1.3712967093020775e-13
Gaussiana 2: A2 = 35.02637784386329, mu2 = 7.872878653441049e-13, sigma2 = 6.779816280558562e-13
Gaussiana 3: A3 = 68.81143330137778, mu3 = 2.008721854221317e-12, sigma3 = 1.2380863193089529e-12
